In [ ]:
#This downloads metagenome assemblies from Segata lab data repository and filters out gyrB sequences 
#using a blast db created from ASVs from this study and GTDB-Tk reference db

In [1]:
import os
import pandas as pd
os.chdir('/Volumes/AHN/captive_ape_microbiome/')

In [2]:
metagenomic_datasets = pd.read_csv('metadata/metagenome_http_to_dnload.txt',header=None)
metagenomic_datasets.columns = ['https']
metagenomic_datasets['filename'] = metagenomic_datasets['https'].apply(lambda x: x.split('/')[-1])
metagenomic_datasets['folder'] = metagenomic_datasets['filename'].apply(lambda x: x.split('.')[0])
metagenomic_datasets.head()


,https,filename,folder
0,https://www.dropbox.com/s/njbwzpazz4a1dy1/Asni...,AsnicarF_2017.tar.bz2,AsnicarF_2017
1,https://www.dropbox.com/s/n6867hhmo12xnwf/Back...,BackhedF_2015.tar.bz2,BackhedF_2015
2,https://www.dropbox.com/s/ac6cpq50pt5mqj6/Beng...,Bengtsson-PalmeJ_2015.tar.bz2,Bengtsson-PalmeJ_2015
3,https://www.dropbox.com/s/uj9cy9914l0cq3x/Brit...,BritoIL_2016.tar.bz2,BritoIL_2016
4,https://www.dropbox.com/s/dtxmz1apdgtf3d1/Cast...,Castro-NallarE_2015.tar.bz2,Castro-NallarE_2015


In [16]:
#create blastdb to filter metagenomic assembly reads
#contains Bacteroidales seqs from gtdbtk taxonomy giving wide taxonomic breadth and
#also Bt gyrB ASVs from present study. It is necessary to include wild ape seqs because
#they are quite divergent.  

In [7]:
%%bash
mkdir results/gyrb_metagenomic
cat results/gyrb_bt_gtdbtk_ref/blast_db/gtdbtk_gyrb_amplicon.fasta \
results/gyrb_all/Bt/ASVs_filtered.fasta > results/gyrb_metagenomic/Bt_ASV_gtdbtk.fasta


mkdir: results/gyrb_metagenomic: File exists


In [23]:
os.chdir('/Volumes/AHN/captive_ape_microbiome/')
folder = 'AsnicarF_2017'
fastas = [f for f in os.listdir(f'results/gyrb_metagenomic/{folder}/') if f.endswith('.fa')]
os.system(f'mkdir -pv results/gyrb_metagenomic/hmmsearch/{folder}')
for fasta in fastas:
    print(fasta)
    
def filter_gyrb_reads(fasta):
    #os.system(f'prodigal -i results/gyrb_metagenomic/{folder}/{fasta} \
    #                     -a results/gyrb_metagenomic/{folder}/{fasta}_genes \
    #                     -d results/gyrb_metagenomic/{folder}/{fasta}_prot')
    os.system(f'hmmsearch --tblout results/gyrb_metagenomic/hmmsearch/{folder}/{fasta}_hmmout.txt \
                ref_seqs/GTDBTK_db/markers/tigrfam/individual_hmms/TIGR01059.HMM \
                results/gyrb_metagenomic/{folder}/{fasta}_prot')
    try:
        hmm_hits = pd.read_csv(f'results/gyrb_metagenomic/hmmsearch/{folder}/{fasta}_hmmout.txt',delim_whitespace=True,header=None,comment='#')
    except:
        print(fasta,'no hits')
        return
    
    hmm_hits = hmm_hits[0] 
    hmm_hits_eval = hmm_hits[0][hmm_hits[4]<float(1e250)]

    #extract genes
    records = SeqIO.parse(f'results/gyrb_metagenomic/{folder}/{fasta}_genes', 'fasta')
    records_hits = [rec for rec in records if rec.id in hmm_hits]
    SeqIO.write_fasta(records_hits,f'results/gyrb_metagenomic/hmmsearch/{folder}/{fasta}_genes','fasta')

    os.system(f'blastx -query results/gyrb_metagenomic/hmmsearch/{folder}/{fasta}_genes \
                       -db results/gyrb_bt_gtdbtk_ref/blast_db/gtdbtk_gyrb_amplicon.faa \
                       -outfmt "7 qseqid sseqid pident length qlen evalue sseq" \
                       -out results/gyrb_metagenomic/blastout/{fasta}_blastout.txt')
    blast_res = pd.read_csv(f'results/gyrb_metagenomic/blastout/{fasta}_blastout.txt',sep='\t',comment='#')

for fasta in fastas:
    filter_gyrb_reads(fasta)

AsnicarF_2017__MV_FEI1_t1Q14.fa
AsnicarF_2017__MV_FEI2_t1Q14.fa
AsnicarF_2017__MV_FEI3_t1Q14.fa
AsnicarF_2017__MV_FEI4_t1Q14.fa
AsnicarF_2017__MV_FEI4_t2Q15.fa
AsnicarF_2017__MV_FEI5_t1Q14.fa
AsnicarF_2017__MV_FEI5_t2Q14.fa
AsnicarF_2017__MV_FEI5_t3Q15.fa
AsnicarF_2017__MV_FEM1_t1Q14.fa
AsnicarF_2017__MV_FEM2_t1Q14.fa
AsnicarF_2017__MV_FEM3_t1Q14.fa
AsnicarF_2017__MV_FEM4_t1Q14.fa
AsnicarF_2017__MV_FEM4_t2Q15.fa
AsnicarF_2017__MV_FEM5_t1Q14.fa
AsnicarF_2017__MV_FEM5_t2Q14.fa
AsnicarF_2017__MV_FEM5_t3Q15.fa
AsnicarF_2017__MV_MIM2_t1M14.fa
AsnicarF_2017__MV_MIM3_t1M14.fa
AsnicarF_2017__MV_MIM4_t2F15.fa
AsnicarF_2017__MV_MIM5_t2M14.fa
AsnicarF_2017__MV_MIM5_t3F15.fa
AsnicarF_2017__MV_FEI1_t1Q14.fa no hits
AsnicarF_2017__MV_FEI2_t1Q14.fa no hits
AsnicarF_2017__MV_FEI3_t1Q14.fa no hits
AsnicarF_2017__MV_FEI4_t1Q14.fa no hits
AsnicarF_2017__MV_FEI4_t2Q15.fa no hits
AsnicarF_2017__MV_FEI5_t1Q14.fa no hits
AsnicarF_2017__MV_FEI5_t2Q14.fa no hits
AsnicarF_2017__MV_FEI5_t3Q15.fa no hits
AsnicarF

In [12]:
for index, row in metagenomic_datasets.iterrows():
    http = row['https']
    filename = row['filename']
    folder = row['folder']
    print(http,filename,folder)
    !scripts/metagenomic_samples/dnload_blast.sh $http $filename $folder
    break
    #if not os.path.exists(f'blastout/{folder}'):
    #    download_metagenomic_dataset_blast_gyrb_seqs(http,filename,folder)
    #else:
    #    print(folder,'already downloaded')

https://www.dropbox.com/s/njbwzpazz4a1dy1/AsnicarF_2017.tar.bz2 AsnicarF_2017.tar.bz2 AsnicarF_2017
AsnicarF_2017
AsnicarF_2017/AsnicarF_2017__MV_FEI1_t1Q14.fa
AsnicarF_2017/AsnicarF_2017__MV_FEI1_t1Q14


Building a new DB, current time: 05/19/2020 16:08:04
New DB name:   /Volumes/AHN/captive_ape_microbiome/results/gyrb_metagenomic/AsnicarF_2017/AsnicarF_2017__MV_FEI1_t1Q14.fa
New DB title:  AsnicarF_2017/AsnicarF_2017__MV_FEI1_t1Q14.fa
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /Volumes/AHN/captive_ape_microbiome/results/gyrb_metagenomic/AsnicarF_2017/AsnicarF_2017__MV_FEI1_t1Q14.fa
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 41591 sequences in 1.36189 seconds.
AsnicarF_2017/AsnicarF_2017__MV_FEI2_t1Q14.fa
AsnicarF_2017/AsnicarF_2017__MV_FEI2_t1Q14


Building a new DB, current time: 05/19/2020 16:08:06
New DB name:   /Volumes/AHN/captive_ape_microbiome/results/gyrb_metagenomic/AsnicarF_2017/AsnicarF_2017__MV_FEI2_t1Q1

AsnicarF_2017/AsnicarF_2017__MV_FEM5_t1Q14.fa
AsnicarF_2017/AsnicarF_2017__MV_FEM5_t1Q14


Building a new DB, current time: 05/19/2020 16:08:54
New DB name:   /Volumes/AHN/captive_ape_microbiome/results/gyrb_metagenomic/AsnicarF_2017/AsnicarF_2017__MV_FEM5_t1Q14.fa
New DB title:  AsnicarF_2017/AsnicarF_2017__MV_FEM5_t1Q14.fa
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /Volumes/AHN/captive_ape_microbiome/results/gyrb_metagenomic/AsnicarF_2017/AsnicarF_2017__MV_FEM5_t1Q14.fa
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 260457 sequences in 7.78004 seconds.
AsnicarF_2017/AsnicarF_2017__MV_FEM5_t2Q14.fa
AsnicarF_2017/AsnicarF_2017__MV_FEM5_t2Q14


Building a new DB, current time: 05/19/2020 16:09:02
New DB name:   /Volumes/AHN/captive_ape_microbiome/results/gyrb_metagenomic/AsnicarF_2017/AsnicarF_2017__MV_FEM5_t2Q14.fa
New DB title:  AsnicarF_2017/AsnicarF_2017__MV_FEM5_t2Q14.fa
Sequence type: Nucleotide
Deleted existing Nucl

0

In [ ]:
	#prep output folders
	mkdir blastout/$folder/
	mkdir fastaout/$folder/
	
	#loop through sample files
	for file in $(cat sample_files/${folder}_files.txt)
	do 	
		#blast search of sample file for gyrb seqs
		echo $file
		makeblastdb -in $file -dbtype nucl
		blastn -query Bif_rep_set_passing.fasta -db $file -outfmt